In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
import random
#import keras_tuner as kt


### Cargue de los datos

In [2]:
# Cargar archivos (rutas relativas desde notebooks/)
train = pd.read_csv("../data/raw/train_data.csv")
test = pd.read_csv("../data/raw/test_data.csv")
sample_submission = pd.read_csv("../data/raw/sampleSubmission.csv")

### Normalización de los datos

In [3]:
# Separar features y target
X = train.drop(columns=["ID", "Bankruptcy"])
y = train["Bankruptcy"]

# Guardar ID del test para el submission
test_ids = test["ID"]
X_test = test.drop(columns=["ID"])

# Dividir training en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

#X_train_scaled[:3]  # Mostrar ejemplo de datos escalados

### Redes Neuronales

In [4]:
# Fijar semillas para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Modelo mejorado
model = models.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(1, activation='sigmoid')
])

# Compilación con tasa de aprendizaje explícita
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=[AUC(name='auc')])


early_stop = EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_val_scaled, y_val),
                    epochs=100,  # subimos el máximo de epochs
                    batch_size=32,
                    callbacks=[early_stop],
                    verbose=1)

# Evaluar en el set de validación
val_predictions = model.predict(X_val_scaled)
auc_score = roc_auc_score(y_val, val_predictions)

# Predecir para el archivo de test
test_predictions = model.predict(X_test_scaled)

Epoch 1/100
52/52 [==============================] - 2s 9ms/step - loss: 0.7758 - auc: 0.5639 - val_loss: 0.5422 - val_auc: 0.6839
Epoch 2/100
52/52 [==============================] - 0s 3ms/step - loss: 0.5662 - auc: 0.6299 - val_loss: 0.4034 - val_auc: 0.7335
Epoch 3/100
52/52 [==============================] - 0s 3ms/step - loss: 0.4596 - auc: 0.5492 - val_loss: 0.3132 - val_auc: 0.7293
Epoch 4/100
52/52 [==============================] - 0s 3ms/step - loss: 0.3488 - auc: 0.6985 - val_loss: 0.2417 - val_auc: 0.6889
Epoch 5/100
52/52 [==============================] - 0s 3ms/step - loss: 0.2799 - auc: 0.6494 - val_loss: 0.1934 - val_auc: 0.7327
Epoch 6/100
52/52 [==============================] - 0s 3ms/step - loss: 0.2235 - auc: 0.6799 - val_loss: 0.1582 - val_auc: 0.8258
Epoch 7/100
52/52 [==============================] - 0s 3ms/step - loss: 0.1800 - auc: 0.7550 - val_loss: 0.1366 - val_auc: 0.8524
Epoch 8/100
52/52 [==============================] - 0s 3ms/step - loss: 0.1615 - a

In [5]:
# Crear DataFrame de submission con las predicciones del modelo (basado en PCA)
submission_df = pd.DataFrame({
    'ID': test_ids,  # Usamos test_ids guardado al inicio
    'Bankruptcy': test_predictions.flatten()
})

# Guardar el archivo correctamente
submission_df.to_csv("../results/submission_scaled1.csv", index=False)

# Verificar
submission_df.head()

,ID,Bankruptcy
0,5,0.008460
1,14,0.007052
2,16,0.014421
3,26,0.001890
4,28,0.020560
